In [2]:
import pandas as pd

from warnings import filterwarnings
filterwarnings("ignore")

In [4]:
df = pd.read_csv("base_address_ner_dataset.csv")
df.head()

,text_part
0,"771001001 кпп по октмо гбу ""жилищник района м..."
1,"5147746350230, окато 45283559000 октмо 453670..."
2,"он. статья 15 адреса, реквизиты и подписи стор..."
3,сударственное бюджетное учреждение гор. москвы...
4,остью «дорстройгрупп» (ооо «дорстройгрупп» юри...


In [5]:
df.shape

(746, 1)

In [25]:
print(df["text_part"][0])

 771001001 кпп по октмо гбу "жилищник района митино" государственные бюджетные учреждения субъектов российской федерации 7733902394 кпп 773301001 по окфс место нахождения (адрес) S125430, город москва, пятницкое шоссе, дом 23E реквизиты сведений об объекте контроля 45000000 инн по окопф форма собственности № п/п дата протокола по октмо 75203 29368873 45367000000 реквизиты документа, содержащего информацию для осуществления


In [30]:
df["text_part"] = df["text_part"].str.replace("S", "S ").str.replace("E", "E ").str.replace("\s\s+", " ")

In [31]:
print(df["text_part"][0])

 771001001 кпп по октмо гбу "жилищник района митино" государственные бюджетные учреждения субъектов российской федерации 7733902394 кпп 773301001 по окфс место нахождения (адрес) S 125430, город москва, пятницкое шоссе, дом 23E  реквизиты сведений об объекте контроля 45000000 инн по окопф форма собственности № п/п дата протокола по октмо 75203 29368873 45367000000 реквизиты документа, содержащего информацию для осуществления


In [32]:
from nltk.tokenize import WordPunctTokenizer

wpt = WordPunctTokenizer()

In [38]:
def prepare_text(text):
    tokens = wpt.tokenize(text)

    mask = []
    is_address = False
    for token in tokens:
        if token == "S":
            mask.append(0)
            is_address = True
            continue
        elif token == "E":
            mask.append(0)
            is_address = False
            continue

        index = 1 if is_address else 0
        mask.append(index)

    df  = pd.DataFrame({"token": tokens, "is_address": mask})
    df.loc[:, "prev_info", ] = df["is_address"].shift(1).fillna(0.0)
    df = df[~df["token"].isin(["S", "E"])]

    hist = 3
    for i in range(1, hist + 1):
        df.loc[:, f"hist_{i}"] = df["token"].shift(i)

    future = 3
    for i in range(1, future + 1):
        df.loc[:, f"future_{i}"] = df["token"].shift(-i)
    df = df.fillna("UNKNOWN")
    return df

In [39]:
prepare_text(text_df["text_part"][0])

,token,is_address,prev_info,hist_1,hist_2,hist_3,future_1,future_2,future_3
0,771001001,0,0.0,UNKNOWN,UNKNOWN,UNKNOWN,кпп,по,октмо
1,кпп,0,0.0,771001001,UNKNOWN,UNKNOWN,по,октмо,гбу
2,по,0,0.0,кпп,771001001,UNKNOWN,октмо,гбу,""""
3,октмо,0,0.0,по,кпп,771001001,гбу,"""",жилищник
4,гбу,0,0.0,октмо,по,кпп,"""",жилищник,района
...,...,...,...,...,...,...,...,...,...
61,",",1,1.0,документа,реквизиты,45367000000,содержащего,информацию,для
62,содержащего,1,1.0,",",документа,реквизиты,информацию,для,осуществления
63,информацию,1,1.0,содержащего,",",документа,для,осуществления,UNKNOWN
64,для,1,1.0,информацию,содержащего,",",осуществления,UNKNOWN,UNKNOWN


In [40]:
text_dfs = []
for i in text_df["text_part"].values:
    text_dfs.append(prepare_text(i))

In [42]:
len(text_dfs)

746

In [43]:
text_dfs = pd.concat(text_dfs)
text_dfs.head()

,token,is_address,prev_info,hist_1,hist_2,hist_3,future_1,future_2,future_3
0,771001001,0,0.0,UNKNOWN,UNKNOWN,UNKNOWN,кпп,по,октмо
1,кпп,0,0.0,771001001,UNKNOWN,UNKNOWN,по,октмо,гбу
2,по,0,0.0,кпп,771001001,UNKNOWN,октмо,гбу,""""
3,октмо,0,0.0,по,кпп,771001001,гбу,"""",жилищник
4,гбу,0,0.0,октмо,по,кпп,"""",жилищник,района
